<a href="https://colab.research.google.com/github/RazvanPorojan/NeuralNet/blob/master/network_debug_mat_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        
    def initialize(self, sizes):
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        print("initialized")

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        print("SGD Started")
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        #print("1.0 === self.cost_derivative(activations[-1], y):%s, sigmoid_prime(zs[-1]).shape:%s, delta.shape:%s" \
              #% (self.cost_derivative(activations[-1], y).shape, sigmoid_prime(zs[-1]).shape, delta.shape))
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        #print("1.1 === delta.shape: %s, activations[-2].transpose().shape: %s" \
              #% (delta.shape, activations[-2].transpose().shape))
        #print("1.2 === nabla_w[-1].shape:%s %s" % (nabla_w[-1].shape,"."))
              
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            #de ce nu am shape-uri diferite pt. layere - layerul de imput nu apare
            #print("2.0 === l:%s" % (l))
            #print("2.1 === self.weights[-l+1].transpose().shape:%s, delta.shape:%s" \
                  #% (self.weights[-l+1].transpose().shape, delta.shape))
            
            #print("2.2 === sp.shape:%s, np.dot(self.weights[-l+1].transpose(), delta).shape):%s" \
                  #% (sp.shape, np.dot(self.weights[-l+1].transpose(), delta).shape)) 

            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            #print("2.3 === np.dot(self.weights[-l+1].transpose(), delta) * sp NEW DELTA: %s %s" \
                 #% (delta.shape, "."))
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

😘😘😘😘

In [1]:
import json
with open('biases0.json', 'r') as myfile:
    biases0=myfile.read()
with open('biases1.json', 'r') as myfile:
    biases1=myfile.read()
with open('weights0.json', 'r') as myfile:
    weights0=myfile.read()
with open('weights1.json', 'r') as myfile:
    weights1=myfile.read()
net.biases[0] = np.asarray(json.loads(biases0))
net.biases[1] = np.asarray(json.loads(biases1))
net.weights[0] = np.asarray(json.loads(weights0))
net.weights[1] = np.asarray(json.loads(weights1))

FileNotFoundError: ignored

In [0]:
net = Network([784, 30, 10])

In [0]:
net.initialize([784, 30, 10])

initialized


In [0]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [0]:
net.SGD(training_data, 8, 10, 3.0, test_data=test_data)

Those are for data export and manipulation

In [0]:
!git clone https://github.com/RazvanPorojan/NeuralNet

In [0]:
!pwd && cd NeuralNet && pwd && cp -r * ../

/content
/content/NeuralNet


In [0]:
!pip install numpy

In [0]:
import numpy as np
aa = np.arange(3).reshape(3,1)
bb = np.arange(5).reshape(1,5)
print(aa)
print(bb)
print(aa.shape)
print(bb.shape)
np.dot(aa,bb)

In [0]:
import numpy as np
aa = np.arange(15).reshape(3,5)
bb = np.arange(5).reshape(5,1)
print(aa.shape)
print(bb.shape)
print(aa)
print(bb)
print("=======")
print(np.dot(aa,bb))

np.dot(aa,bb).shape

In [0]:
new_data = []
for tr_item in test_data:
    nimage, label = tr_item
    image = nimage.tolist()
    new_data.append((image, label))

In [0]:
new_tr_data = []
for tr_item in training_data:
    nimage, label = tr_item
    image = nimage.tolist()
    new_tr_data.append((image, label))

In [0]:
#not working with original, working with new data
import scipy.io as sio
import numpy as np

sio.savemat('new_test_data.mat',{"new_data":new_data})